# OpenCV SSD

> ## Video Object Detection

* Pretrained 모델 파일 로드 후 OpenCV 기반 Object Detection 수행 

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# I. Video File Download

* matrix.mp4 다운로드

In [ ]:
!wget https://raw.githubusercontent.com/rusita-ai/pyData/master/image/matrix.mp4

--2023-04-08 01:56:06--  https://raw.githubusercontent.com/rusita-ai/pyData/master/image/matrix.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 386522 (377K) [application/octet-stream]
Saving to: ‘matrix.mp4’

matrix.mp4          100%[===================>] 377.46K  --.-KB/s    in 0.03s   

2023-04-08 01:56:06 (10.7 MB/s) - ‘matrix.mp4’ saved [386522/386522]



* matrix.mp4 확인

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open('/content/matrix.mp4', 'r+b').read()

encoded = base64.b64encode(video)

HTML(data='''<video width = "70%" controls>
                <source src = "data:video/mp4;base64,{0}" type = "video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

# II. Video Frame Information

> ## 1) VideoCapture( )

* cv2.CAP_PROP_FRAME_COUNT

In [ ]:
import cv2

video_input_path = '/content/matrix.mp4'

cap = cv2.VideoCapture(video_input_path)
frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('Frame Count:', frame_cnt)

Frame Count: 134


> ## 2) VideoWriter( )

* Encoding Codec and FPS and Size 

In [ ]:
video_input_path = '/content/matrix.mp4'
video_output_path = '/content/matrix_ssd.mp4'

cap = cv2.VideoCapture(video_input_path)

codec = cv2.VideoWriter_fourcc(*'XVID')

vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))) 

vid_fps = cap.get(cv2.CAP_PROP_FPS )
    
vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size) 

print('FPS Count:', vid_fps)
print('Video Size:', vid_size)

FPS Count: 30.0
Video Size: (640, 360)


# III. Object Detection API Download

> ## 1) Pretrained Model Download

* https://github.com/opencv/opencv/wiki/TensorFlow-Object-Detection-API
* http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2017_11_17.tar.gz

* ssd_inception_v2_coco_2017_11_17.tar.gz 다운로드

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2017_11_17.tar.gz

--2023-04-08 01:56:07--  http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2017_11_17.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.12.128, 2607:f8b0:400c:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.12.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278126337 (265M) [application/x-tar]
Saving to: ‘ssd_inception_v2_coco_2017_11_17.tar.gz’

ssd_inception_v2_co 100%[===================>] 265.24M   145MB/s    in 1.8s    

2023-04-08 01:56:09 (145 MB/s) - ‘ssd_inception_v2_coco_2017_11_17.tar.gz’ saved [278126337/278126337]



* ssd_inception_v2_coco_2017_11_17.tar.gz 압축해제

In [ ]:
!tar -xvf ssd_inception_v2_coco_2017_11_17.tar.gz

ssd_inception_v2_coco_2017_11_17/
ssd_inception_v2_coco_2017_11_17/model.ckpt.index
ssd_inception_v2_coco_2017_11_17/model.ckpt.meta
ssd_inception_v2_coco_2017_11_17/frozen_inference_graph.pb
ssd_inception_v2_coco_2017_11_17/model.ckpt.data-00000-of-00001
ssd_inception_v2_coco_2017_11_17/saved_model/
ssd_inception_v2_coco_2017_11_17/saved_model/saved_model.pb
ssd_inception_v2_coco_2017_11_17/saved_model/variables/
ssd_inception_v2_coco_2017_11_17/checkpoint


* 결과 확인
 - frozen_inference_graph.pb

In [ ]:
!ls -l /content/ssd_inception_v2_coco_2017_11_17

total 201116
-rw-r----- 1 275875 5000        77 Nov 18  2017 checkpoint
-rw-r----- 1 275875 5000 101987441 Nov 18  2017 frozen_inference_graph.pb
-rw-r----- 1 275875 5000 100214372 Nov 18  2017 model.ckpt.data-00000-of-00001
-rw-r----- 1 275875 5000     18073 Nov 18  2017 model.ckpt.index
-rw-r----- 1 275875 5000   3701354 Nov 18  2017 model.ckpt.meta
drwxr-x--- 3 275875 5000      4096 Nov 18  2017 saved_model


> ## 2) Model Configuration Download

* https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/ssd_inception_v2_coco_2017_11_17.pbtxt

In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/ssd_inception_v2_coco_2017_11_17.pbtxt

--2023-04-08 01:56:18--  https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/ssd_inception_v2_coco_2017_11_17.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117525 (115K) [text/plain]
Saving to: ‘ssd_inception_v2_coco_2017_11_17.pbtxt’

ssd_inception_v2_co 100%[===================>] 114.77K  --.-KB/s    in 0.02s   

2023-04-08 01:56:18 (5.35 MB/s) - ‘ssd_inception_v2_coco_2017_11_17.pbtxt’ saved [117525/117525]



# IV. Model and Configuration load

* cv2.dnn.readNetFromTensorflow()

In [ ]:
cv_net = cv2.dnn.readNetFromTensorflow('/content/ssd_inception_v2_coco_2017_11_17/frozen_inference_graph.pb',
                                       '/content/ssd_inception_v2_coco_2017_11_17.pbtxt')

# V. Class ID Setting

* COCO Dataset

> ## 1) OpenCV - TensorFlow

* SSD

In [ ]:
labels_to_names = {1:'person',2:'bicycle',3:'car',4:'motorcycle',5:'airplane',6:'bus',7:'train',8:'truck',9:'boat',10:'traffic light',
                   11:'fire hydrant',12:'street sign',13:'stop sign',14:'parking meter',15:'bench',16:'bird',17:'cat',18:'dog',19:'horse',20:'sheep',
                   21:'cow',22:'elephant',23:'bear',24:'zebra',25:'giraffe',26:'hat',27:'backpack',28:'umbrella',29:'shoe',30:'eye glasses',
                   31:'handbag',32:'tie',33:'suitcase',34:'frisbee',35:'skis',36:'snowboard',37:'sports ball',38:'kite',39:'baseball bat',40:'baseball glove',
                   41:'skateboard',42:'surfboard',43:'tennis racket',44:'bottle',45:'plate',46:'wine glass',47:'cup',48:'fork',49:'knife',50:'spoon',
                   51:'bowl',52:'banana',53:'apple',54:'sandwich',55:'orange',56:'broccoli',57:'carrot',58:'hot dog',59:'pizza',60:'donut',
                   61:'cake',62:'chair',63:'couch',64:'potted plant',65:'bed',66:'mirror',67:'dining table',68:'window',69:'desk',70:'toilet',
                   71:'door',72:'tv',73:'laptop',74:'mouse',75:'remote',76:'keyboard',77:'cell phone',78:'microwave',79:'oven',80:'toaster',
                   81:'sink',82:'refrigerator',83:'blender',84:'book',85:'clock',86:'vase',87:'scissors',88:'teddy bear',89:'hair drier',90:'toothbrush',
                   91:'hair brush'}


# VI. Object Detection

* 약 2분

* 총 Frame 별로 iteration 하면서 Object Detection 수행. 개별 frame별로 단일 이미지 Object Detection과 유사 

In [ ]:
%%time

import time

green_color = (0, 255, 0)
red_color = (0, 0, 255)

while True:

    hasFrame, img_frame = cap.read()
    if not hasFrame:
        print('No more Frames')
        break

    rows = img_frame.shape[0]
    cols = img_frame.shape[1]

    cv_net.setInput(cv2.dnn.blobFromImage(img_frame, swapRB = True))
    
    start= time.time()

    cv_out = cv_net.forward()
    frame_index = 0

    for detection in cv_out[0, 0, :, :]:
        score = float(detection[2])
        class_id = int(detection[1])

        if score > 0.5:
            left = detection[3] * cols
            top = detection[4] * rows
            right = detection[5] * cols
            bottom = detection[6] * rows

            caption = "{}: {:.4f}".format(labels_to_names[class_id], score)

            cv2.rectangle(img_frame, (int(left), int(top)), (int(right), int(bottom)), color = green_color, thickness = 2)
            cv2.putText(img_frame, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1)
    print('Detection Time:', round(time.time() - start, 2), '초')
    vid_writer.write(img_frame)

vid_writer.release()
cap.release()   

Detection Time: 1.28 초
Detection Time: 0.68 초
Detection Time: 0.7 초
Detection Time: 0.69 초
Detection Time: 0.69 초
Detection Time: 0.65 초
Detection Time: 0.65 초
Detection Time: 0.63 초
Detection Time: 0.69 초
Detection Time: 0.62 초
Detection Time: 0.5 초
Detection Time: 0.66 초
Detection Time: 0.71 초
Detection Time: 0.68 초
Detection Time: 0.79 초
Detection Time: 0.75 초
Detection Time: 0.82 초
Detection Time: 0.79 초
Detection Time: 0.69 초
Detection Time: 0.69 초
Detection Time: 0.66 초
Detection Time: 0.66 초
Detection Time: 0.39 초
Detection Time: 0.4 초
Detection Time: 0.4 초
Detection Time: 0.38 초
Detection Time: 0.39 초
Detection Time: 0.39 초
Detection Time: 0.4 초
Detection Time: 0.39 초
Detection Time: 0.37 초
Detection Time: 0.39 초
Detection Time: 0.38 초
Detection Time: 0.39 초
Detection Time: 0.39 초
Detection Time: 0.38 초
Detection Time: 0.39 초
Detection Time: 0.38 초
Detection Time: 0.38 초
Detection Time: 0.4 초
Detection Time: 0.38 초
Detection Time: 0.38 초
Detection Time: 0.48 초
Detection Time: 0

# VII. Define Object Detection Function

> ## 1) get_detected_img( ) 정의

In [ ]:
import time

def get_detected_img(cv_net, img_array, score_threshold, use_copied_array = True, is_print = True):
    
    rows = img_array.shape[0]
    cols = img_array.shape[1]
    
    draw_img = None
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    cv_net.setInput(cv2.dnn.blobFromImage(img_array, swapRB = True))
    
    start = time.time()
    cv_out = cv_net.forward()
    
    green_color = (0, 255, 0)
    red_color = (0, 0, 255)

    for detection in cv_out[0, 0, :, :]:
        score = float(detection[2])
        class_id = int(detection[1])
        # score_threshold 이상만 추출
        if score > score_threshold:
            left = detection[3] * cols
            top = detection[4] * rows
            right = detection[5] * cols
            bottom = detection[6] * rows

            caption = "{}: {:.4f}".format(labels_to_names[class_id], score)
            print(caption)

            cv2.rectangle(draw_img, (int(left), int(top)), (int(right), int(bottom)), color = green_color, thickness = 2)
            cv2.putText(draw_img, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1)
    if is_print:
        print('Detection Time:',round(time.time() - start, 2), '초')

    return draw_img

> ## 2) do_deteced_video( ) 정의

In [ ]:
def do_detected_video(cv_net, input_path, output_path, score_threshold, is_print):
    
    cap = cv2.VideoCapture(input_path)

    codec = cv2.VideoWriter_fourcc(*'XVID')

    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_fps = cap.get(cv2.CAP_PROP_FPS)

    vid_writer = cv2.VideoWriter(output_path, codec, vid_fps, vid_size) 

    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print('Frame Count:', frame_cnt)

    green_color=(0, 255, 0)
    red_color=(0, 0, 255)
    while True:
        hasFrame, img_frame = cap.read()
        if not hasFrame:
            print('No more Frames')
            break
        
        img_frame = get_detected_img(cv_net, img_frame, score_threshold = score_threshold, use_copied_array = False, is_print = is_print)
        
        vid_writer.write(img_frame)
    # end of while loop

    vid_writer.release()
    cap.release()

> ## 3) korea2.mp4 Download

In [ ]:
!wget https://raw.githubusercontent.com/rusita-ai/pyData/master/image/korea.mp4

--2023-04-08 01:57:23--  https://raw.githubusercontent.com/rusita-ai/pyData/master/image/korea.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6859828 (6.5M) [application/octet-stream]
Saving to: ‘korea.mp4’

korea.mp4           100%[===================>]   6.54M  --.-KB/s    in 0.08s   

2023-04-08 01:57:23 (79.5 MB/s) - ‘korea.mp4’ saved [6859828/6859828]



* korea.mp4 확인

In [ ]:
import io
import base64
from IPython.display import HTML

video = io.open('/content/korea.mp4', 'r+b').read()

encoded = base64.b64encode(video)

HTML(data='''<video width = "70%" controls>
                <source src = "data:video/mp4;base64,{0}" type = "video/mp4"/>
             </video>'''.format(encoded.decode('ascii')))

Output hidden; open in https://colab.research.google.com to view.

> ## 4) do_deteced_video( ) 실행

* 약 20분

In [ ]:
%%time

do_detected_video(cv_net, '/content/korea.mp4', '/content/korea_ssd.mp4', 0.3, False)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
tv: 0.6903
person: 0.4614
tv: 0.4145
person: 0.3380
person: 0.3059
tv: 0.8221
person: 0.4205
tv: 0.3753
person: 0.3344
tv: 0.7832
person: 0.4371
person: 0.3578
person: 0.3359
tv: 0.3148
tv: 0.7948
person: 0.4529
tv: 0.3341
person: 0.4362
person: 0.4263
person: 0.3863
tv: 0.3842
person: 0.3232
tv: 0.3091
tv: 0.4792
person: 0.4368
person: 0.4391
person: 0.4175
tv: 0.3656
tv: 0.3225
tv: 0.3028
person: 0.4931
person: 0.3845
fork: 0.3344
fork: 0.5001
person: 0.4702
person: 0.4304
person: 0.5274
person: 0.3628
person: 0.3350
person: 0.5110
person: 0.3314
person: 0.7009
person: 0.3452
scissors: 0.5574
person: 0.4311
person: 0.4216
scissors: 0.7192
person: 0.5492
person: 0.3410
scissors: 0.6034
spoon: 0.3486
scissors: 0.7197
person: 0.3029
scissors: 0.4647
person: 0.4468
person: 0.3209
person: 0.3128
scissors: 0.6824
spoon: 0.3103
toothbrush: 0.4842
scissors: 0.7493
scissors: 0.3268
bed: 0.3334
person: 0.3077
person: 0.3603
person: 0.3338
person: 0.3049
pers

# 
# 
# 
# The End
# 
# 
# 